In [3]:
%use s2 , algoquant, krangl, lets-plot

val begDate = LocalDateTime.of(2018, 1, 1, 0, 0)
val endDate = LocalDateTime.of(2020, 1, 1, 0, 0)
val simInterval = LocalDateTimeInterval.of(begDate, endDate)

// rebalancing period and data window
val rebPer = Period.ofMonths(3)
val timeWin = Period.ofYears(2)

// simulation inputs
val tickers = s2.dataFrame(
    "ticker-metadata",
    "AAPL", "MSFT", "TSLA", "GOOGL", "AMZN", "FB", "XOM", "APA", "BHI", "EOG"
)

// risk averse
val riskA = 1.0

// estimation parameters
val linearShrinkageMeanEstimatorParam = 0.5
val linearShrinkageCovEstimatorParam = 0.5

// get rid of multiple tickers
val ticGroup = tickers.groupBy("idtable")
val sepTic = ticGroup.groups()[0]

// create cache for portfolio
val permTics = sepTic["permaticker"]
val tics = sepTic["ticker"]
val cache = BufferCache<List<Product>>()

cache.add(LocalDateTime.MIN, emptyList())
val products: MutableList<Product> = ArrayList(tickers.nrow)
for (index in 0 until sepTic.nrow) {
    products.add(
        QuandlProduct(
            tics[index] as String?,
            permTics[index].toString() as String?,
            Currencies.USD
        )
    )
}
cache.add(begDate, products)

// create data provider
val dbCacheProvider = DbCacheProvider(simInterval, cache)

// long-only constraint
val stocksCache = BufferCache<List<String>>()
stocksCache.add(begDate, tics.asStrings().filterNotNull())
val constraint = MarkowitzLongOnlyConstraint().addLongOnlyPositionLimit(stocksCache)

// create optimization algo
val opt_algo = MarkowitzDynamicConstraintOptimizationAlgorithm(riskA, constraint)

// estimation method
opt_algo.meanEstimator = MeanAnnualizingWrapper(LinearShrinkageMeanEstimator(linearShrinkageMeanEstimatorParam), rebPer)
opt_algo.covarianceEstimator = CovarianceAnnualizingWrapper(LinearShrinkageCovarianceEstimator(linearShrinkageCovEstimatorParam), rebPer)
// run simulation
val sim = OptimizationAlgorithmComparisonSimulation(
    dbCacheProvider.allProducts(simInterval),
    dbCacheProvider,
    dbCacheProvider,
    rebPer,
    timeWin
)
// get benchmark data and risk free rate
sim.setRiskFreeRateLookup(DbRiskFreeRateLookup().newRiskFreeRateLookup(simInterval))
sim.setBenchmarkLookupFactory("SP500", BenchmarkLookupFactory("sp500"))
sim.addAlgorithm("markowitz", opt_algo)
val result = sim.run(simInterval)
println("success")
// print performances
val values: MutableList<Any> = ArrayList()
val report = result.get("markowitz")!!.report()
report.measureKeys().forEach {
    values.add(it)
    values.add(report.getPerformance(it))
}

val df = dataFrameOf("Measure", "Value")(values)
df.print(title = "Performance Measures", maxRows = df.nrow, maxDigits = 4, rowNumbers = false)
// print nav
val date = report.navCache().map { it.time().toInstant(ZoneOffset.UTC) }.toList()
val nav = report.navCache().map { it.data() }.toList()
val navHistory = mapOf("Date" to date, "NAV" to nav)
var p = lets_plot(navHistory)
p += geom_line(color = "dark_green") { x = "Date"; y = "NAV" } + scale_x_datetime()
p + ggsize(800, 450)


java.lang.RuntimeException: table 'ticker-metadata' not found
dev.nm.s2.data.DataCommand.void(fb:32)
dev.nm.s2.data.DataCommand.void(fb:168)
dev.nm.s2.data.DataCommand.dataFrame(fb:1)
Line_42_jupyter.<init>(Line_42.jupyter.kts:12)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:96)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:41)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eva